## Hierarchical Architecture

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph, MessageGraph

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [3]:
unique_id = "Hierarchical Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [4]:
# from langsmith import Client

# client = Client()

In [5]:
class PromptReview(BaseModel):
    """Review of the prompt"""
    # prompt: str = Field(description="Most recent prompt")
    feedback: str = Field(description="Feedback on the most recent prompt")

class WorkerPrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])

class WorkerAgent:
    """
    Worker Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: WorkerPrinciples, temp: float = 1.0, model: str = "gpt-4o"):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = SystemMessage(content=f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}
You must use your expertise and core principles to guide all your thinking. You must speak only as an expert in your field.""")        
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )

    def review_prompt(self, prompt: str, additional_info: str) -> PromptReview:
        """
        Generates a review of the prompt.
        """
        template = """Your task is to think outside the box and provide feedback on the prompt below with creative recommendations on how to improve it in light of your core principles:
{prompt}

Your feedback must be less than 100 words so think carefully about the most critical aspects of the prompt that need improvement.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow when providing feedback and recommendations:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {position}. 
2. Identify the most critical aspects of the prompt that need improvement. 
3. Think outside the box to provide creative feedback with recommendations on how to improve the prompt in light of your core principles.
4. Ensure that your feedback is less than 100 words.
5. Ensure that your feedback follows the strict guidelines provided above.
6. Submit your feedback.

Return only your feedback in JSON format below:

{{
    "feedback": "Feedback on the original prompt"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptReview)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "prompt": prompt, "additional_info": additional_info})
                # Validate the output before returning
                if completion.feedback:
                    return completion
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")

In [6]:
class Workforce(BaseModel):
    """Details of workforce generated by the leader agent."""
    positions: List[str] = Field(description="Positions of the workers in the workforce")
    core_principles: List[List[str]] = Field(description="Core principles of the workers in the workforce")
    

class FeedbackSummary(BaseModel):
    """Summary of feedback from the worker agents."""
    feedback_summary: str = Field(description="Collated and summarised feedback from the workforce")


class ApprovalDecision(BaseModel):
    """Decision of the leader agent to approve or disapprove the prompt."""
    approved: bool = Field(description="Decision to approve or disapprove the prompt")
    explanation: str = Field(description="A detailed explanation of why the prompt was approved or not")


class TeamLeaderAgent:
    """
    TeamLeaderAgent class defining an agent that manages a team of worker agents to help optimise prompts.
    """

    def __init__(self, team: str, team_role: str, prompt: str, additional_info: str = None, temp: float = 1.0, model: str = "gpt-4o", workforce: List[WorkerAgent] = None):
        self.team = team
        self.team_role = team_role
        self.system_message = SystemMessage(content=f"""You are an experienced {team} team leader with expertise in leading a team with the role: {team_role}. 
You must use your expertise to guide all your thinking. You must speak only as an expert in your field.""")
        self.prompt = prompt
        self.additional_info = additional_info
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.workforce = workforce

    def leader_feedback(self, prompt: str, feedback: list) -> str:
        """
        LeaderAgent to decide the next worker or to finish.
        """
        # All workers except the current worker
        template = """Your task is to refine and summarise the feedback below:
{feedback}

You must capture the important aspects of the feedback and recommendations provided by your team.
Your summary must be actionable and precise, providing clear recommendations for improvements to the prompt below:
{prompt}

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow when summarising feedback and recommendations:
- DO NOT suggest modifying existing restrictions.
- DO NOT suggest modifying or removing negations.
- DO NOT suggest adding, modifying or removing placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.

Your summarisation process should be as follows:
1. Read the feedback carefully as an experienced {team} team leader.
2. Identify the important aspects of the feedback and recommendations provided by your team.
3. Refine and summarise the feedback and recommendations into a concise and actionable summary.
4. Ensure that your summary adheres to the strict guidelines provided above.
5. Ensure that your summary is actionable and precise, providing clear recommendations for improvements to the prompt.
6. Submit your summary.

Return only the feedback summary in JSON format below:

{{
    "feedback_summary": "Feedback summary",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=FeedbackSummary)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["feedback", "prompt", "additional_info", "team"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"feedback": feedback, "prompt": prompt, "additional_info": self.additional_info, "team": self.team})
                break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        return output.feedback_summary
       
    def get_feedback(self, state):
        """
        Get feedback from wokers. Feedback collected concurrently.
        """
        with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
            futures = [executor.submit(worker.review_prompt, state['prompt'], self.additional_info) for worker in self.workforce]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
            # feedback = [result[-1].content for result in results]
            feedback = [result.feedback for result in results]

        summary = self.leader_feedback(state["prompt"], feedback)

        return summary
    
    def get_approval(self, prompt: str):
        """
        Agent to approve or reject the prompt.
        """
        template = """Your task is to review the prompt below and decide whether or not it should be approved in light of your team and team role:
{prompt}

Return the boolean value "True" if you approve and think the prompt already optimal in the aspects your team:, {members} specialises in. 
Return the boolean value "False" if you disapprove and think the prompt could be improved with the insights of your team.
You must provide a detailed explanation of why you approve or disapprove of the prompt.

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert in {team}
2. Determine whether the prompt will be effective in instructing the model to perform the desired task.
3. If you believe the prompt is effective, approve it for use by the model.
4. Submit your decision.
    
Return only the approval decision and explanation in JSON format below:

{{
    "approved": "True/False"
    "explanation": "Detailed explanation of why the prompt was approved or not"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=ApprovalDecision)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "members", "additional_info", "team"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"prompt": prompt, "members": [worker.position for worker in self.workforce], "additional_info": self.additional_info, "team": self.team})
                # Validate the output before returning
                if completion.approved is not None and completion.explanation:
                    return completion.approved
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")

In [7]:
class Domain(BaseModel):
    """Domain of the prompt."""
    domain: str = Field(description="Domain of the prompt")
    description: str = Field(description="Description of the domain team role")


class RouteDecision(BaseModel):
    """Decision on the next worker to process."""
    next: str = Field(description="The next team to process")


class UpdatedPrompt(BaseModel):
    """Updated prompt based on feedback from the team leader."""
    updated_prompt: str = Field(description="Updated prompt based on feedback")


class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates worker agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, additional_info: str = None, temp: float = 1.0, model: str = "gpt-4o", team_leaders: List[TeamLeaderAgent] = None):
        self.system_message = SystemMessage(content=f"""You are an experienced senior AI professional. You specialise in prompt engineering. 
You have in-depth knowledge of large language models and prompt engineering best practices. Use this knowledge to inform all your decisions.""")
        self.base_prompt = base_prompt
        self.prompt = base_prompt
        self.additional_info = additional_info
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.team_leaders = team_leaders
        self.team_roles_dict = {team_leader.team: team_leader.team_role for team_leader in self.team_leaders}
        self.iterations = 0
    
    def run_approval(self, prompt: str) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(team_leader.get_approval, prompt) for team_leader in self.team_leaders]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: dict) -> RouteDecision:
        """
        LeaderAgent to decide the next worker or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state["prompt"])
        print("Approval results:", approval_results)
        if all(approval_results) or (self.iterations > 10):
            return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]} 
        else:
            self.iterations += 1
            disapproved_team_leaders = [team_leader for i, team_leader in enumerate(self.team_leaders) if not approval_results[i]]
            options = [team_leaders.team for team_leaders in disapproved_team_leaders]
            # shuffle options to avoid positional bias
            random.shuffle(options)
            # options = ["FINISH"] + members
            template = """Your task is to review the prompt below and decide the next team to provide feedback:
{prompt}

Below are details of what the prompt is expected to instruct the model to do: 
{additional_info}

The details of the teams and their roles are as follows: 
{team_roles}

Select one of the following teams to provide feedback on the prompt: 
{options}

Think carefully about pairing the aspects of the prompt that need improvement and how they relate to the expertise of each team.
If you think multiple aspects of the prompt need improvement, select the most suitable team to provide feedback on the most critical aspects of the prompt.

Return only the next team name to process in JSON format below:

{{
    "next": "Next team",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
            pydantic_parser = PydanticOutputParser(pydantic_object=RouteDecision)
            prompt_template = PromptTemplate(
                system_message=self.system_message,
                template=template,
                input_variables=["prompt", "additional_info", "team_roles", "options"],
                partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
            )
            chain = prompt_template | self.llm | pydantic_parser
            for _ in range(3):
                try:
                    output = chain.invoke({"prompt": state["prompt"], "additional_info": self.additional_info, "team_roles": str(self.team_roles_dict), "options": str(options)})
                    break
                except Exception as e:
                    print("Exception occurred:", e)
                    continue
            return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}

    def update_prompt(self, prompt: str, feedback: str, history) -> str:
        """
        Updates the prompt with the feedback from the worker agent.
        """
        template = """Your task is to implement feedback to improve the prompt below:
{prompt}

The feedback is as follows:
{feedback} 

You must also consider the discussion history below prior to making changes to the prompt to ensure you do not repeat any mistakes: 
{history}

Below are details of what the prompt is expected to instruct the model to do:
{additional_info}

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN UNCHANGED.

Your update process should be as follows:
1. Read the prompt and feedback carefully.
2. Review the discussion history to ensure you do not repeat any mistakes.
3. Implement the feedback provided to help the prompt better achieve what is expected.
4. Ensure that your update follows the strict guidelines provided above.
5. Submit your updated prompt.

Return only the updated prompt in JSON format:

{{
    "prompt": "Updated prompt"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=UpdatedPrompt)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "feedback", "history", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": prompt, "feedback": feedback, "history": history, "additional_info": self.additional_info})
                if output.updated_prompt:
                    return output.updated_prompt
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", output)
            except Exception as e:
                print("Exception occurred:", e)
                continue

    def construct_team_graph(self):
        """
        Constructs a graph of team leader agents.
        """
        # The agent state is the input to each node in the graph
        class TeamState(TypedDict):
            # The annotation tells the graph that new messages will always be added to the current states
            messages: Sequence[BaseMessage]
            prompt: str
            next: str

        def team_node(state, team_leader):
            try:
                feedback = team_leader.get_feedback(state)
                updated_prompt = self.update_prompt(state["prompt"], feedback, state["messages"])
                # updated_prompt = output[-1].content 
                self.prompt = updated_prompt
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Feedback: {feedback}", name=team_leader.team),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to leader with the most recent prompt
                print(f"Parsing failed for {team_leader.team}: {e}")
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Error: Parsing failed for {team_leader} - {e}", name=team_leader.team),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],                    
                    "prompt": state["prompt"],
                    "next": "leader",
                }

        workflow = StateGraph(TeamState)
        for team_leader in self.team_leaders:
            # Create a node for each team leader agent
            # team_leader.construct_worker_graph()
            node = functools.partial(team_node, team_leader=team_leader)
            workflow.add_node(team_leader.team, node)
        workflow.add_node("leader", self.leader_decision)

        members = [team_leader.team for team_leader in self.team_leaders]
        for member in members:
            # We want our workers to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("leader", lambda x: x["next"], conditional_map)
        # Finally, add entrypoint
        workflow.set_entry_point("leader")
        graph = workflow.compile()
        
        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of worker agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"{self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "leader",
        }

        # Construct the graph
        graph = self.construct_team_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        # Run the graph
        for s in graph.stream(
            initial_state,
            {"recursion_limit": 50}
            ):
            if "__end__" not in s:
                print(s)
                print("----")
                continue

        # if not os.path.exists("prompt_history_hierarchical.json"):
        #     with open("prompt_history_hierarchical.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_hierarchical.json", "r") as f:
        #     data = json.load(f)
        #     data.append(self.prompt_history)
            
        # with open("prompt_history_hierarchical.json", "w") as f:
        #     json.dump(data, f, indent=4)
                
        return s

In [8]:
# Prompt design team members
conciseness_and_clarity_principles = WorkerPrinciples([
    "Always write clear and concise prompts",
    "Always use simple and direct language to communicate ideas",
    "Always consider the task requirements when formatting and structuring prompts",
])
conciseness_and_clarity_expert = WorkerAgent("Conciseness and Clarity Expert", conciseness_and_clarity_principles)

contextual_relevance_principles = WorkerPrinciples([
    "Always provide context to help the model understand the task",
    "Always consider the context in which prompts will be used",
])
contextual_relevance_expert = WorkerAgent("Contextual Relevance Expert", contextual_relevance_principles)

task_alignment_principles = WorkerPrinciples([
    "Always ensure that prompts align with the task requirements",
    "Always tailor instructions to the task to guide the model",
    "Always consider the expected output of the model",
])
task_alignment_expert = WorkerAgent("Task Alignment Expert", task_alignment_principles)

example_demonstration_principal = WorkerPrinciples([
    "Always provide examples to help the model understand the task",
    "Always ensure examples are relevant and clear",
    "Always demonstrate the expected output of the model",
])
example_demonstration_expert = WorkerAgent("Example Demonstration Expert", example_demonstration_principal)

avoiding_bias_principles = WorkerPrinciples([
    "Always avoid bias in prompts",
    "Always consider the ethical implications of prompts",
])
avoiding_bias_expert = WorkerAgent("Avoiding Bias Expert", avoiding_bias_principles)

incremental_prompting_principles = WorkerPrinciples([
    "Always provide clear step-by-step instructions to guide the model",
    "Always consider the complexity of the task when providing incremental instructions",
])
incremental_prompting_expert = WorkerAgent("Incremental Prompting Expert", incremental_prompting_principles)

programming_logic_principles = WorkerPrinciples([
    "Always ensure that prompts are logically structured, similar to programming logic",
    "Always consider the logical flow of instructions in prompts",
    "Always consider the usefulness of programming logic to the task",
])
programming_logic_expert = WorkerAgent("Programming Logic Expert", programming_logic_principles)

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
# Domain team members
mathematics_principles = WorkerPrinciples([
    "Always think with a mathematical mindset",
    "Always use mathetical operators correctly",
    "Always adhere to mathematical rules",
])
mathematician = WorkerAgent("Mathematician", mathematics_principles)

word_problem_solving_principles = WorkerPrinciples([
    "Always pay attention to the keywords in the problem",
    "Always approach problems systematically",
    "Always consider multiple approaches to solving problems",
])
word_problem_solver = WorkerAgent("Word Problem Solver", word_problem_solving_principles)


In [10]:
base_prompt = "{content}. Please output your answer at the end as ##<your answer (arabic numerals)>."
additional_info = "- Solve the math word problem.\n- Output the answer at the end as ##<your answer (arabic numerals)> with no spaces or units."

prompt_design_team = TeamLeaderAgent(
    "Prompt Design",
    "Formulate prompts and instructions to elicit high-quality responses from large language models. The focus is to utilise input from prompt experts to improve the prompt.",
    base_prompt,
    additional_info,
    workforce=[
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        avoiding_bias_expert,
        incremental_prompting_expert,
        programming_logic_expert,
    ]
)
domain_team = TeamLeaderAgent(
    "Mathematics",
    "Formulate prompts and instructions help large language models solve mathematical problems. The focus is to utilise input from domain experts to improve the prompt.",
    base_prompt,
    additional_info,
    workforce=[
        mathematician,
        word_problem_solver,
    ]
)

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
    team_leaders=[prompt_design_team, domain_team]
)

# leader_agent.generate_teams()
# print teams and members
print("Teams and members:")
for team_leader in leader_agent.team_leaders:
    print(f"{team_leader.team} team:")
    print(f"Team Role: {team_leader.team_role}")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Core Principles: {worker.core_principles}")
    print("----")

Teams and members:
Prompt Design team:
Team Role: Formulate prompts and instructions to elicit high-quality responses from large language models. The focus is to utilise input from prompt experts to improve the prompt.
Position: Conciseness and Clarity Expert, Core Principles: - Always write clear and concise prompts
- Always use simple and direct language to communicate ideas
- Always consider the task requirements when formatting and structuring prompts
Position: Contextual Relevance Expert, Core Principles: - Always provide context to help the model understand the task
- Always consider the context in which prompts will be used
Position: Task Alignment Expert, Core Principles: - Always ensure that prompts align with the task requirements
- Always tailor instructions to the task to guide the model
- Always consider the expected output of the model
Position: Example Demonstration Expert, Core Principles: - Always provide examples to help the model understand the task
- Always ensure e

In [11]:
result = leader_agent.optimise_prompt()

Exception occurred: Invalid json output: {
    "approved": False,
    "explanation": "The prompt provided is not clear enough in its instructions. As a Mathematician and Word Problem Solver, I noticed that the instruction to solve the math word problem and output the answer in a specific format (##<your answer (arabic numerals)>= with no spaces or units) could be misinterpreted. For precise communication, it should specify that the answer should be given directly as a number enclosed in double hash symbols with no surrounding text, e.g., ##42##. This ensures clarity and maintains consistency in the format required. Therefore, I disapprove the current version of the prompt and suggest it includes this explicit example."
}
Exception occurred: Invalid json output: ```json
{
    "approved": False,
    "explanation": "The prompt requires some improvements to meet the criteria set by the team's specializations. While the prompt is concise and clear, it lacks detailed instructions and example

In [12]:
result["leader"]["prompt"]

"{content}. **Include all necessary reasoning steps before the final answer to ensure comprehensiveness.** **Detail all steps and state any assumptions made.** **Avoid explanatory text, units, or extra spaces.** **Output the answer in the format: ##<your answer (arabic numerals)>**. Example of correctly formatted output: '##42'. Example of reasoning steps: 'Simplify the expression 2 + 2: 2 + 2 = 4, therefore the answer is ##4.'"

### Concurrent Runs

In [ ]:
base_prompt = "Classify the sentence as positive or negative: {content}"
additional_info = "This is a classification task with only two classes: positive and negative."

leader_agent_1 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
print("Teams and members:")
for team_leader in leader_agent_1.team_leaders:
    print(f"{team_leader.team} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Role: {worker.role}, Function: {worker.function}")
    print("----")

leader_agent_2 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
print("Teams and members:")
for team_leader in leader_agent_2.team_leaders:
    print(f"{team_leader.team} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Role: {worker.role}, Function: {worker.function}")
    print("----")

leader_agent_3 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
print("Teams and members:")
for team_leader in leader_agent_3.team_leaders:
    print(f"{team_leader.team} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Role: {worker.role}, Function: {worker.function}")
    print("----")

Teams and members:
prompt writing team:
Position: Prompt Engineering Specialist, Role: Design and refine prompts for optimal performance, Function: Develop and test various prompt structures
Position: Sentiment Analysis Expert, Role: Ensure accurate classification of sentiment, Function: Review and fine-tune sentiment detection mechanisms
Position: Natural Language Processing Scientist, Role: Implement advanced NLP techniques for prompt optimization, Function: Apply state-of-the-art NLP models and methods to enhance prompt effectiveness
----
generic team:
Position: Natural Language Processing Specialist, Role: Develop and fine-tune NLP models, Function: Implement and refine machine learning models for text classification
Position: Sentiment Analysis Expert, Role: Design sentiment classification algorithms, Function: Design and validate sentiment analysis methodologies to accurately classify text as positive or negative
Position: Prompt Optimization Engineer, Role: Optimize prompt desig

In [33]:

# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine prompts to create a more effective prompt.
You have in-depth knowledge regarding large language models and their associated architectures, as well as prompt engineering best practices."""

    template = """I am going to tip $300K for a better prompt!
Given the prompts below, your task is to merge the best parts of each prompt to create the most effective prompt.
Carefully consider the strengths of each prompt and how they can be combined to create a better prompt.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
- Incremental prompting
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Ensure that your answer is unbiased.

Prompts: {results}

Return only the next worker to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

Classify the sentence as positive or negative: {content}
----
Classify the sentiment of the following sentence into one of two categories: 'positive' or 'negative'. Focus on the context and nuances of the content provided to determine the sentiment accurately. Provide your classification based on the overall sentiment conveyed by the sentence. Sentence: {content}
----
Classify the sentence provided in {content} as either 'positive' or 'negative' based on its sentiment. For this task, 'positive' sentiment indicates expressions of happiness, approval, or any favorable emotions, while 'negative' sentiment denotes expressions of sadness, disapproval, or any unfavorable emotions. Ensure the classification is strictly 'positive' or 'negative' and do not consider any ambiguous sentiments.
----


In [34]:
print(final_result)

Classify the sentiment of the following sentence into one of two categories: 'positive' or 'negative'. Focus on the context and nuances of the content provided to determine the sentiment accurately. For this task, 'positive' sentiment indicates expressions of happiness, approval, or any favorable emotions, while 'negative' sentiment denotes expressions of sadness, disapproval, or any unfavorable emotions. Ensure the classification is strictly 'positive' or 'negative' and do not consider any ambiguous sentiments. Sentence: {content}
